# test a model

Test youy rmodel right after training in a kubeflow pipeline, or run this function independently.  In addition, the plotting components in **[test_classifier.py]()** can also be run independently.

In [1]:
import mlrun
import os
import numpy as np
mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

## parameters

In [2]:
CODE_BASE          = '/User/repos/functions'

MODEL_FILE         = '/User/mlrun/models/lgb-classifier.pkl'

TARGET_DATA_PATH   = '/User/mlrun/splitter'
XTEST_FILE         = '/User/mlrun/splitter/xtest.pqt'
YTEST_FILE         = '/User/mlrun/splitter/ytest.pqt'

## run tests

In [3]:
yaml_name = os.path.join(CODE_BASE, 'evaluation', 'test-classifier.yaml')
if not os.path.isfile(yaml_name):
    testfn = mlrun.code_to_function(
        kind='job', 
        image='yjbds/mlrun-ds:latest',
        filename=os.path.join(CODE_BASE, 'evaluation', 'test-classifier.py'))
    testfn.build_config(base_image='yjbds/mlrun-ds:latest', commands=[])

    testfn.export(os.path.join(CODE_BASE, 'evaluation', 'test-classifier.yaml'))

In [4]:
testfn = mlrun.import_function(
    os.path.join(CODE_BASE, 'evaluation', 'test-classifier.yaml')
).apply(mlrun.mount_v3io())

In [5]:
testfn.deploy(skip_deployed=True, with_mlrun=False)

'ready'

In [6]:
task = mlrun.NewTask()
task.with_params(
    model=MODEL_FILE,
    xtest=XTEST_FILE,
    ytest=YTEST_FILE,
    target_path=TARGET_DATA_PATH)

In [7]:
tsk_run = testfn.run(task, handler='test_model')

[mlrun] 2020-01-26 13:19:07,552 starting run test_model uid=b2dff34b7d184eb89aac9e919764e9c4  -> http://mlrun-api:8080
[mlrun] 2020-01-26 13:19:07,644 Job is running in the background, pod: test-model-4qkft
[mlrun] 2020-01-26 13:19:17,924 Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/mlrun/runtimes/local.py", line 174, in exec_from_params
    val = handler(*args_list)
  File "main.py", line 68, in test_model
    ypred_probs = clf.predict_proba(xtest)[:, 1]
  File "/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py", line 858, in predict_proba
    pred_leaf, pred_contrib, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py", line 658, in predict
    % (self._n_features, n_features))
ValueError: Number of features of the model must match the input. Model n_features_ is 20 and input n_features is 28 


[mlrun] 2020-01-26 13:19:17,941 exec error - Number of features of the model must match the input. Model n_features_ is 

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...64e9c4,0,Jan 26 13:19:14,error,test-classifier,host=test-model-4qkftkind=jobowner=admin,,model=/User/mlrun/models/lgb-classifier.pkltarget_path=/User/mlrun/splitterxtest=/User/mlrun/splitter/xtest.pqtytest=/User/mlrun/splitter/ytest.pqt,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run b2dff34b7d184eb89aac9e919764e9c4  , !mlrun logs b2dff34b7d184eb89aac9e919764e9c4 
[mlrun] 2020-01-26 13:19:26,852 run executed, status=error
runtime error: Number of features of the model must match the input. Model n_features_ is 20 and input n_features is 28 


RunError: Number of features of the model must match the input. Model n_features_ is 20 and input n_features is 28 